# LinkedIn API

In [1]:
!pip install linkedin_api

In [2]:
from linkedin_api import Linkedin
import pandas as pd
import time
import getpass

In [3]:
def login():
    '''Logging in with credentials to use API'''

    # Setting maximum try limit to 5 log-in attempts
    max_attempts = 5
    attempt = 0
    api = None

    while attempt < max_attempts:
        try:
            user = input('LinkedIN ID: ')

            # To protect User privacy
            passwd = getpass.getpass("Enter your password: ")

            # Logging in in API
            api = Linkedin(user, passwd)
            return api
        except Exception:
            print(f"Login failed. Error: Kindly check your credentials")
            attempt += 1
            if attempt < max_attempts:
                print(f"Remaining attempts: {max_attempts - attempt}")
            else:
                print("Maximum login attempts reached. Exiting.")
                return None


def search_name():
    '''Conducting Search - Firstname Lastname'''
    # Getting name
    name = input('Enter Name : ')
    if name == '':
        print('No name provided! Search will be conducted as per LinkedIn Blank search based on connections!\n')
        return None,None
    try:
        #If full name is provided
        fname,lname = name.split(' ')
        return fname,lname
    except ValueError:
        # If only firstname is provided
        print('Proceeding without a lastname')
        fname = name
        return fname,None


def profile_information_of_top_10_profiles(fname,lname=None):
    '''Getting unique URN IDs for each profile and getting profile data based on each urn_id'''
    
    # Getting URN ID from search_people function of API
    search_results = [urn['urn_id'] for urn in api.search_people(keyword_first_name=fname,
                                                                keyword_last_name = lname,
                                                                   limit=100)[:10]]
    
    #Getting Profile Information from URN_ID from get_profile function
    profile_info = [pd.DataFrame([api.get_profile(urn_id = id)]) for id in search_results]
    return profile_info


def save_data(profiles):
    '''Saving data in a .csv format'''
    try:
        # Getting dataframe with all top entires
        data = pd.concat(profiles,axis=0)

         # Replacing empty lists by NaN
        data = data.applymap(lambda x: None if isinstance(x, list) and len(x) == 0 else x)

        # Getting import information in the first indexes for better readability
        desired_column_order = ['profile_id','firstName','lastName','summary','education','experience','headline','industryName']
        desired_column_order.extend(col for col in data.columns if col not in desired_column_order)
        data = data[desired_column_order]
        
        # Saving data
        data.to_csv('Results')
        print('Data Saved!!!')
        return data  #Just for review purpose
        
    except Exception:
        print('No user found!\nNo data Saved!')
        return None
    

## Example use case of LinkedIn API

In [4]:
start_time = time.time()

api = login()
if api:
    print('Congratulations, you are permitted to use LinkedIN API\n\n\n')
    first_name,last_name = search_name()
    profiles_info = profile_information_of_top_10_profiles(fname=first_name,lname=last_name)
    data = save_data(profiles_info)

    end_time = time.time()
    total_runtime = end_time - start_time
    print(f"Total runtime: {total_runtime:.2f} seconds")
    
else:
    exit()

Congratulations, you are permitted to use LinkedIN API



Data Saved!!!
Total runtime: 79.93 seconds


In [5]:
data

,profile_id,firstName,lastName,summary,education,experience,headline,industryName,locationName,student,...,member_urn,public_id,languages,publications,certifications,volunteer,honors,projects,birthDate,industryUrn
0,ACoAADnFpgkBL8XnRJ1DpTz3pgSb5xjnmZpSdus,Ram,Chaitanya,NaN,[{'entityUrn': 'urn:li:fs_education:(ACoAADnFp...,"[{'locationName': 'Hazira', 'entityUrn': 'urn:...",Student at Visvesvaraya National Institute of ...,NaN,India,False,...,urn:li:member:969254409,ram-chaitanya-149727230,None,None,None,None,None,None,NaN,NaN
0,ACoAAAZgHZIB2M_zj2-9YUV6Q7o-qyQO_JnQAn8,Ram,Chaitanya,"Currently, I am building a sustainability plat...",[{'entityUrn': 'urn:li:fs_education:(ACoAAAZgH...,[{'entityUrn': 'urn:li:fs_position:(ACoAAAZgHZ...,Advancing sustainable living by empowering eco...,Renewables & Environment,India,False,...,urn:li:member:106962322,ramchaitanya1,None,None,None,"[{'role': 'Digital Marketing Consultant', 'com...",[{'description': '- Awarded ‘Best Account Stra...,None,"{'month': 8, 'day': 31}",urn:li:fs_industry:144
0,ACoAAAGRBsABM_ArPx2tArep-jCtFEmu9ZYq2Y8,P. Vijaya Ram,Chaitanya,I started my career in 2006 with a simple visi...,[{'entityUrn': 'urn:li:fs_education:(ACoAAAGRB...,"[{'locationName': 'Vishakhapatnam, Andhra Prad...",Happy with work,Information Technology & Services,India,False,...,urn:li:member:26281664,vijayaramchaitanya,"[{'name': 'English'}, {'name': 'HIndhi'}, {'na...",None,None,None,None,None,NaN,urn:li:fs_industry:96
0,ACoAACO4y8YB13SpfU-lIE-Sq2dP0m_loc_LD74,Ram,Chaitanya,3 yrs 4 months,[{'entityUrn': 'urn:li:fs_education:(ACoAACO4y...,"[{'locationName': 'India', 'entityUrn': 'urn:l...",Service Operations Senior IP Analyst,Consumer Goods,India,False,...,urn:li:member:599313350,ram-chaitanya-a32860148,None,None,None,None,None,None,"{'month': 6, 'day': 28}",urn:li:fs_industry:25
0,ACoAAC-3hKYBrEKuemzcHL_xYojXMX1VU7qEAxk,Ram,chaitanya,A highly disciplined and hardworking individua...,[{'projects': ['urn:li:fs_project:(ACoAAC-3hKY...,None,Student at SRM University (Sri Ramaswamy Memor...,Computer Software,NaN,False,...,urn:li:member:800556198,ram-chaitanya-a061301a4,None,None,None,None,None,[{'occupation': 'urn:li:fs_education:(ACoAAC-3...,NaN,urn:li:fs_industry:4
0,ACoAAD4g5G8BVENX4fa84tb8wEtUXYy_CJv9FYc,Ram,Chaitanya,NaN,[{'entityUrn': 'urn:li:fs_education:(ACoAAD4g5...,None,"Student at Amrita Vishwa Vidyapeetham, Bengalu...",Higher Education,India,False,...,urn:li:member:1042343023,ram-chaitanya-,None,None,None,None,None,None,"{'month': 3, 'day': 10}",urn:li:fs_industry:68
0,ACoAADGRBzIBBPuGvUIprtOwTO5uRxW8ioPIKpk,Siva ram Krishna,Chaitanya,NaN,[{'entityUrn': 'urn:li:fs_education:(ACoAADGRB...,"[{'locationName': 'Visakhapatnam, Andhra Prade...",Student at Gandhi Institute of Technology & Ma...,Logistics & Supply Chain,India,False,...,urn:li:member:831588146,siva-ram-krishna-chaitanya-1825b71b2,None,None,None,None,None,None,NaN,urn:li:fs_industry:116
0,ACoAADl_EcYBrKq1FdcyH59NVk2H3ImJzw6gAxg,Venkat Ram,Chaitanya,NaN,[{'entityUrn': 'urn:li:fs_education:(ACoAADl_E...,None,Student at Vellore Institute of Technology,Primary/Secondary Education,India,False,...,urn:li:member:964628934,venkat-ram-chaitanya-40607622b,None,None,"[{'authority': 'Microsoft', 'name': 'Start the...",None,None,None,NaN,urn:li:fs_industry:67
